CDS Depth Estimation

In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR



## This is the model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    # torch.Size([8, 3, 448, 576])
    # torch.Size([8, 448, 576])
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if args.dry_run:
                break


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


def main():
    # Training settings
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--epochs', type=int, default=14, metavar='N',
                        help='number of epochs to train (default: 14)')
    parser.add_argument('--lr', type=float, default=1.0, metavar='LR',
                        help='learning rate (default: 1.0)')
    parser.add_argument('--gamma', type=float, default=0.7, metavar='M',
                        help='Learning rate step gamma (default: 0.7)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--no-mps', action='store_true', default=False,
                        help='disables macOS GPU training')
    parser.add_argument('--dry-run', action='store_true', default=False,
                        help='quickly check a single pass')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')
    parser.add_argument('--save-model', action='store_true', default=False,
                        help='For Saving the current Model')
    args = parser.parse_args()
    use_cuda = not args.no_cuda and torch.cuda.is_available()
    use_mps = not args.no_mps and torch.backends.mps.is_available()

    torch.manual_seed(args.seed)

    if use_cuda:
        device = torch.device("cuda")
    elif use_mps:
        device = torch.device("mps")
    else:
        device = torch.device("cpu")

    train_kwargs = {'batch_size': args.batch_size}
    test_kwargs = {'batch_size': args.test_batch_size}
    if use_cuda:
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
    dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        scheduler.step()

    if args.save_model:
        torch.save(model.state_dict(), "mnist_cnn.pt")


if __name__ == '__main__':
    main()


In [4]:
import glob
import torch
import matplotlib.pyplot as plt

from dataset.base_dataset import get_dataset
dataset_kwargs = {'dataset_name': 'nyudepthv2', 'data_path': './data/'}
# dataset_kwargs['crop_size'] = (args.crop_h, args.crop_w)

train_dataset = get_dataset(**dataset_kwargs)
val_dataset = get_dataset(**dataset_kwargs, is_train=False)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8,
                                            shuffle=True, num_workers=0, 
                                            pin_memory=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False,
                                             pin_memory=True)

device = torch.device('cpu')
for batch_idx, batch in enumerate(train_loader):
    input_RGB = batch['image'].to(device)
    depth_gt = batch['depth'].to(device)
    filename = batch['filename']
    print(filename, input_RGB)
    print(input_RGB.size())
    print(depth_gt.size())
    break


<class 'dataset.nyudepthv2.nyudepthv2'>
Dataset: NYU Depth V2
# of train images: 24231
<class 'dataset.nyudepthv2.nyudepthv2'>
Dataset: NYU Depth V2
# of test images: 654
['living_room_0040_rgb_00325.jpg', 'playroom_0006_rgb_00108.jpg', 'living_room_0055_rgb_00014.jpg', 'living_room_0019_rgb_00007.jpg', 'bedroom_0014_rgb_00067.jpg', 'bedroom_0034_rgb_00066.jpg', 'bedroom_0082_rgb_00001.jpg', 'furniture_store_0002a_rgb_00060.jpg'] tensor([[[[1.0000, 1.0000, 1.0000,  ..., 0.7098, 0.7098, 0.7098],
          [1.0000, 1.0000, 1.0000,  ..., 0.7098, 0.7098, 0.7098],
          [1.0000, 1.0000, 1.0000,  ..., 0.7216, 0.7216, 0.7176],
          ...,
          [0.5255, 0.5216, 0.5216,  ..., 0.5725, 0.5725, 0.5804],
          [0.5176, 0.5176, 0.5137,  ..., 0.5725, 0.5725, 0.5804],
          [0.5098, 0.5137, 0.5098,  ..., 0.5686, 0.5686, 0.5686]],

         [[1.0000, 1.0000, 1.0000,  ..., 0.6314, 0.6314, 0.6314],
          [1.0000, 1.0000, 1.0000,  ..., 0.6314, 0.6314, 0.6314],
          [1.0000, 1.

In [2]:
## Loading and processing the dataset

In [3]:
from dataset.base_dataset import get_dataset

In [4]:
from dataset.base_dataset import get_dataset

dataset_kwargs = {'dataset_name': 'nyudepthv2', 'data_path': './data/'}
# dataset_kwargs['crop_size'] = (args.crop_h, args.crop_w)

train_dataset = get_dataset(**dataset_kwargs)
val_dataset = get_dataset(**dataset_kwargs, is_train=False)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8,
                                            shuffle=True, num_workers=0, 
                                            pin_memory=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False,
                                             pin_memory=True)





<class 'dataset.nyudepthv2.nyudepthv2'>
Dataset: NYU Depth V2
# of train images: 24231
<class 'dataset.nyudepthv2.nyudepthv2'>
Dataset: NYU Depth V2
# of test images: 654


In [6]:
device = torch.device('cpu')
for batch_idx, batch in enumerate(train_loader):
    input_RGB = batch['image'].to(device)
    depth_gt = batch['depth'].to(device)
    filename = batch['filename']
    print(filename, input_RGB)
    break


['home_office_0006_rgb_00146.jpg', 'study_room_0005a_rgb_00045.jpg', 'cafe_0001a_rgb_00014.jpg', 'kitchen_0052_rgb_00025.jpg', 'bedroom_0069_rgb_00118.jpg', 'bookstore_0001j_rgb_00130.jpg', 'study_room_0005b_rgb_00009.jpg', 'kitchen_0019b_rgb_00037.jpg'] tensor([[[[0.1255, 0.0549, 0.0118,  ..., 0.1490, 0.1451, 0.1490],
          [0.2706, 0.1961, 0.1333,  ..., 0.1529, 0.1451, 0.1490],
          [0.4157, 0.3569, 0.3098,  ..., 0.1529, 0.1451, 0.1451],
          ...,
          [0.1686, 0.1608, 0.1569,  ..., 0.1569, 0.1373, 0.1176],
          [0.1686, 0.1608, 0.1529,  ..., 0.1373, 0.1137, 0.0980],
          [0.1686, 0.1647, 0.1569,  ..., 0.1216, 0.1020, 0.0941]],

         [[0.1765, 0.0980, 0.0510,  ..., 0.1294, 0.1255, 0.1294],
          [0.3412, 0.2549, 0.1922,  ..., 0.1333, 0.1255, 0.1294],
          [0.5020, 0.4314, 0.3804,  ..., 0.1373, 0.1255, 0.1255],
          ...,
          [0.1608, 0.1569, 0.1490,  ..., 0.1216, 0.1137, 0.1020],
          [0.1686, 0.1608, 0.1529,  ..., 0.1098, 0.09

In [6]:
device = torch.device('cpu')
# model.to(device)
for batch_idx, batch in enumerate(val_loader):
        input_RGB = batch['image'].to(device)
        depth_gt = batch['depth'].to(device)
        filename = batch['filename']
        print(filename, input_RGB)

['bathroom_rgb_00045.jpg'] tensor([[[[0.0039, 0.0000, 0.0000,  ..., 0.0039, 0.0000, 0.0000],
          [0.0000, 0.0078, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0039, 0.0039, 0.0000,  ..., 0.0000, 0.0039, 0.0039],
          [0.0039, 0.0039, 0.0000,  ..., 0.0000, 0.0039, 0.0039],
          [0.0039, 0.0078, 0.0039,  ..., 0.0000, 0.0039, 0.0039]],

         [[0.0078, 0.0000, 0.0078,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0118, 0.0039,  ..., 0.0000, 0.0000, 0.0000],
          [0.0039, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0039],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0039,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0157, 0.0078, 0.0039,  ..., 0.0000, 0.0000, 0.0000],
          [0.0078, 0.0196, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.01

In [ ]:
def train(train_loader, model, criterion_d, log_txt, optimizer, device, epoch, args):    
    global global_step
    model.train()
    depth_loss = logging.AverageMeter()
    half_epoch = args.epochs // 2
    iterations = len(train_loader)
    result_lines = []
    for batch_idx, batch in enumerate(train_loader):      
        global_step += 1

        if global_step < iterations * half_epoch:
            current_lr = (args.max_lr - args.min_lr) * (global_step /
                                            iterations/half_epoch) ** 0.9 + args.min_lr
        else:
            current_lr = max(args.min_lr, (args.min_lr - args.max_lr) * (global_step /
                                            iterations/half_epoch - 1) ** 0.9 + args.max_lr)

        for param_group in optimizer.param_groups:
            param_group['lr'] = current_lr*param_group['lr_scale'] if 'swin' in args.backbone else current_lr

        input_RGB = batch['image'].to(device)
        depth_gt = batch['depth'].to(device)

        preds = model(input_RGB)

        optimizer.zero_grad()
        loss_d = criterion_d(preds['pred_d'].squeeze(), depth_gt)
        depth_loss.update(loss_d.item(), input_RGB.size(0))
        loss_d.backward()
        
        if args.pro_bar:
            logging.progress_bar(batch_idx, len(train_loader), args.epochs, epoch,
                                ('Depth Loss: %.4f (%.4f)' %
                                (depth_loss.val, depth_loss.avg)))

        if batch_idx % args.print_freq == 0:
            result_line = 'Epoch: [{0}][{1}/{2}]\t' \
                'Loss: {loss}, LR: {lr}\n'.format(
                    epoch, batch_idx, iterations,
                    loss=depth_loss.avg, lr=current_lr
                )
            result_lines.append(result_line)
            print(result_line)
        optimizer.step()

    with open(log_txt, 'a') as txtfile:
        txtfile.write('\nEpoch: %03d - %03d' % (epoch, args.epochs))
        for result_line in result_lines:
            txtfile.write(result_line)   

    return loss_d

In [ ]:
def main():
    # Training settings
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--epochs', type=int, default=14, metavar='N',
                        help='number of epochs to train (default: 14)')
    parser.add_argument('--lr', type=float, default=1.0, metavar='LR',
                        help='learning rate (default: 1.0)')
    parser.add_argument('--gamma', type=float, default=0.7, metavar='M',
                        help='Learning rate step gamma (default: 0.7)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--no-mps', action='store_true', default=False,
                        help='disables macOS GPU training')
    parser.add_argument('--dry-run', action='store_true', default=False,
                        help='quickly check a single pass')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')
    parser.add_argument('--save-model', action='store_true', default=False,
                        help='For Saving the current Model')
    args = parser.parse_args()
    use_cuda = not args.no_cuda and torch.cuda.is_available()
    use_mps = not args.no_mps and torch.backends.mps.is_available()

    torch.manual_seed(args.seed)

    if use_cuda:
        device = torch.device("cuda")
    elif use_mps:
        device = torch.device("mps")
    else:
        device = torch.device("cpu")

    train_kwargs = {'batch_size': args.batch_size}
    test_kwargs = {'batch_size': args.test_batch_size}
    if use_cuda:
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
    dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        scheduler.step()

    if args.save_model:
        torch.save(model.state_dict(), "mnist_cnn.pt")


if __name__ == '__main__':
    main()

In [7]:
class SelfAttention(Module):
    "Self attention layer for `n_channels`."
    def __init__(self, n_channels):
        self.query,self.key,self.value = [self._conv(n_channels, c) for c in (n_channels//8,n_channels//8,n_channels)]
        self.gamma = nn.Parameter(tensor([0.]))

    def _conv(self,n_in,n_out):
        return ConvLayer(n_in, n_out, ks=1, ndim=1, norm_type=NormType.Spectral, act_cls=None, bias=False)

    def forward(self, x):
        #Notation from the paper.
        size = x.size()
        x = x.view(*size[:2],-1)
        f,g,h = self.query(x),self.key(x),self.value(x)
        beta = F.softmax(torch.bmm(f.transpose(1,2), g), dim=1)
        o = self.gamma * torch.bmm(h, beta) + x
        return o.view(*size).contiguous()

NameError: name 'Module' is not defined